In [3]:
%%pyspark 

items_raw = spark.read.option("header","true").load(path="abfss://data-container@supermarket.dfs.core.windows.net/raw_data/items.csv",format="csv")

StatementMeta(sparkpool, 0, 4, Finished, Available)

In [11]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import *

StatementMeta(sparkpool, 0, 12, Finished, Available)

In [8]:
loss_raw = spark.read.option("header","true").load(path="abfss://data-container@supermarket.dfs.core.windows.net/raw_data/loss.csv",format="csv")
w_price_raw = spark.read.option("header","true").load(path="abfss://data-container@supermarket.dfs.core.windows.net/raw_data/w_price.csv",format="csv")
sales_raw = spark.read.option("header","true").load(path="abfss://data-container@supermarket.dfs.core.windows.net/raw_data/sales.csv",format="csv")

StatementMeta(sparkpool, 0, 9, Finished, Available)

In [14]:
sales = sales_raw.withColumn('Date',col('Date').cast(DateType()))\
.withColumn('Quantity Sold (kilo)',col('Quantity Sold (kilo)').cast(FloatType()))\
.withColumn('Unit Selling Price (RMB/kg)',col('Unit Selling Price (RMB/kg)').cast(FloatType()))


StatementMeta(sparkpool, 0, 15, Finished, Available)

In [15]:
sales = sales.withColumn('Month',month(sales.Date))\
.withColumn("Price",col("Quantity Sold (kilo)")*col("Unit Selling Price (RMB/kg)"))\
.withColumn("Year",year(col('Date')))

StatementMeta(sparkpool, 0, 16, Finished, Available)

In [16]:
loss = loss_raw.withColumn('Loss Rate (%)',col('Loss Rate (%)').cast(FloatType()))

StatementMeta(sparkpool, 0, 17, Finished, Available)

In [17]:
w_price = w_price_raw.withColumn('Date',col('Date').cast(DateType()))\
    .withColumn('Wholesale Price (RMB/kg)',col('Wholesale Price (RMB/kg)').cast(FloatType()))

StatementMeta(sparkpool, 0, 18, Finished, Available)

In [19]:
sales.groupBy("Year").agg(sum("Price").alias("Total sales per year"))\
    .sort(desc('Total sales per year')) \
    .show()

StatementMeta(sparkpool, 0, 20, Finished, Available)

+----+--------------------+
|Year|Total sales per year|
+----+--------------------+
|2021|  1100362.6499556378|
|2022|  1036772.4026289582|
|2020|   669529.2706266977|
|2023|   563102.1486261189|
+----+--------------------+



In [20]:
sales.groupBy("Year","Month").agg(sum("Price").alias("Total sales per Month")).sort(asc('Year'),asc('Month')) \
    .show()

StatementMeta(sparkpool, 0, 21, Finished, Available)

+----+-----+---------------------+
|Year|Month|Total sales per Month|
+----+-----+---------------------+
|2020|    7|   122550.29299824312|
|2020|    8|     133930.331336841|
|2020|    9|   106520.83309704065|
|2020|   10|   122685.28258185089|
|2020|   11|    84574.69156730175|
|2020|   12|    99267.83904542029|
|2021|    1|   135735.11432659626|
|2021|    2|   178817.90269818902|
|2021|    3|    92091.74080651999|
|2021|    4|     74633.4320570603|
|2021|    5|    68115.36016271263|
|2021|    6|    66851.88533959538|
|2021|    7|    83136.95215059817|
|2021|    8|   108694.55542507768|
|2021|    9|    91217.39817206562|
|2021|   10|    84693.81243920326|
|2021|   11|    57670.31753855944|
|2021|   12|   58704.178839460015|
|2022|    1|    114624.3679151833|
|2022|    2|    80479.23535914719|
+----+-----+---------------------+
only showing top 20 rows



In [21]:
sales.groupBy("Item Code").agg(sum("Price").alias("Total sell per day"))\
    .sort(desc('Total sell per day')) \
    .show(10)

StatementMeta(sparkpool, 0, 22, Finished, Available)

+-----------+------------------+
|  Item Code|Total sell per day|
+-----------+------------------+
|  1.029E+14| 3265918.570337724|
| 1.0695E+14| 77040.40110439062|
|1.06972E+14|10654.000120162964|
|1.06959E+14| 7983.700159549713|
| 1.0693E+14| 4808.800060033798|
|1.06956E+14|1403.1000137329102|
|1.06973E+14| 879.9000182151794|
|1.06958E+14| 718.5000233650208|
|1.06974E+14|244.00000023841858|
|1.06932E+14|             115.5|
+-----------+------------------+



In [23]:
org_price = sales.join(w_price,sales["Item Code"]==w_price["Item Code"]).select(sales.Date,'Quantity Sold (kilo)','Wholesale Price (RMB/kg)','Price')

StatementMeta(sparkpool, 0, 24, Finished, Available)

In [24]:
prices = org_price.withColumn('original_price',col("Quantity Sold (kilo)")*col("Wholesale Price (RMB/kg)"))\
    .withColumn("Profit",col("Price")-col("original_price"))

StatementMeta(sparkpool, 0, 25, Finished, Available)

In [33]:
prices = prices.withColumn("Year",year(prices.Date))

StatementMeta(sparkpool, 0, 34, Finished, Available)

In [42]:
prices.select('Year','Profit').groupBy('Year').agg(sum('Profit').alias('Total Profit')).sort(desc('Total Profit')).show()

StatementMeta(sparkpool, 0, 43, Cancelled, Waiting)

In [40]:
prices.show()

StatementMeta(sparkpool, 0, 41, Finished, Available)

+----------+--------------------+------------------------+------+--------------+-----------+----+
|      Date|Quantity Sold (kilo)|Wholesale Price (RMB/kg)| Price|original_price|     Profit|Year|
+----------+--------------------+------------------------+------+--------------+-----------+----+
|2020-07-01|               0.396|                    4.48|3.0096|       1.77408|  1.2355199|2020|
|2020-07-01|               0.396|                    6.45|3.0096|        2.5542|     0.4554|2020|
|2020-07-01|               0.396|                    18.0|3.0096|         7.128| -4.1183996|2020|
|2020-07-01|               0.396|                     3.2|3.0096|        1.2672|  1.7423999|2020|
|2020-07-01|               0.396|                    1.53|3.0096|    0.60587996|    2.40372|2020|
|2020-07-01|               0.396|                    3.12|3.0096|       1.23552|  1.7740799|2020|
|2020-07-01|               0.396|                   12.72|3.0096|       5.03712|   -2.02752|2020|
|2020-07-01|        